Импорты

In [276]:
#Импортирование библиотек
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
from collections import Counter
import sklearn
from sklearn.model_selection import GridSearchCV, train_test_split, ShuffleSplit, KFold
import xgboost as xgb
from sklearn.metrics import classification_report, confusion_matrix, log_loss
import py7zr
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb
from sklearn.metrics import f1_score

Типа загружаю данные и разбиваю на 2 выборки

In [2]:
data = pd.read_csv('contest_train.csv.zip')

In [3]:
data.head(5)

,ID,TARGET,FEATURE_0,FEATURE_1,FEATURE_2,FEATURE_3,FEATURE_4,FEATURE_5,FEATURE_6,FEATURE_7,...,FEATURE_250,FEATURE_251,FEATURE_252,FEATURE_253,FEATURE_254,FEATURE_255,FEATURE_256,FEATURE_257,FEATURE_258,FEATURE_259
0,0d1c880d23ff018,1,2.0,2.0,0.0,0.0,0.0,0.0,0.0,-114.527812,...,39.0,2.0,223.118594,290.015143,1.0,1.0,0.0,1.0,2.0,2.0
1,1579ba37fd82c1f,0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,329.828334,...,25.0,82.0,479.616045,611.651594,0.0,1.0,0.0,1.0,1.0,1.0
2,790086f721c7f5e,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.427338,...,119.0,0.0,352.268014,564.573421,1.0,1.0,0.0,1.0,1.0,1.0
3,b0a2078cf33c226,0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,23.621397,...,1.0,0.0,410.524985,421.653876,1.0,0.0,0.0,1.0,2.0,2.0
4,d0c46717b9cb323,0,4.0,3.0,1.0,0.0,1.0,0.0,0.0,677.745861,...,21.0,0.0,439.171356,476.189288,1.0,1.0,0.0,1.0,1.0,1.0


In [4]:
data.describe()

,TARGET,FEATURE_0,FEATURE_1,FEATURE_2,FEATURE_3,FEATURE_4,FEATURE_5,FEATURE_6,FEATURE_7,FEATURE_8,...,FEATURE_250,FEATURE_251,FEATURE_252,FEATURE_253,FEATURE_254,FEATURE_255,FEATURE_256,FEATURE_257,FEATURE_258,FEATURE_259
count,24521.000000,24521.000000,24521.000000,24521.000000,24521.0,24521.000000,24521.000000,24521.000000,24521.000000,23377.000000,...,24521.000000,24521.000000,24521.000000,24521.000000,24521.000000,24521.000000,24521.0,24521.000000,24521.00000,24521.000000
mean,0.352677,5.438196,1.337833,0.056401,0.0,0.141226,0.000163,0.011378,80.762084,278.787390,...,41.057379,36.403205,307.827071,408.052350,0.611965,0.640390,0.0,1.138575,1.18580,1.184699
std,0.592092,14.533334,1.826413,0.230698,0.0,0.348262,0.012771,0.106061,256.605853,1206.454609,...,105.000146,105.872566,259.669681,349.968987,0.487312,0.479896,0.0,0.373638,0.40809,0.401288
min,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,-171.709797,-297.065679,...,-1256.000000,-667.000000,-1531.063435,-328.361649,0.000000,0.000000,0.0,1.000000,0.00000,1.000000
25%,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,-9.809139,-21.011023,...,0.000000,0.000000,129.829124,179.694989,0.000000,0.000000,0.0,1.000000,1.00000,1.000000
50%,0.000000,1.000000,1.000000,0.000000,0.0,0.000000,0.000000,0.000000,3.673798,4.169065,...,6.000000,0.000000,265.472906,326.928038,1.000000,1.000000,0.0,1.000000,1.00000,1.000000
75%,1.000000,5.000000,2.000000,0.000000,0.0,0.000000,0.000000,0.000000,27.367809,37.939371,...,39.000000,10.000000,408.197249,533.731611,1.000000,1.000000,0.0,1.000000,1.00000,1.000000
max,2.000000,385.000000,39.000000,1.000000,0.0,1.000000,1.000000,1.000000,3147.341440,9963.242114,...,1775.000000,3194.000000,4202.905396,4440.534979,1.000000,1.000000,0.0,3.000000,3.00000,3.000000


In [6]:
y = data.iloc[:,1]
x=data.iloc[:,2:]

In [7]:
#меняю пропуски на median
x=x.fillna(x.median())
#pd.isnull(x)
#x.FEATURE_8

In [8]:
# тут ещё 1 метод медианы
# Create an imputer object with a median filling strategy
#imputer = Imputer(strategy='median')
# Train on the training features
#imputer.fit(train_features)
# Transform both training data and testing data
#X = imputer.transform(train_features)
#X_test = imputer.transform(test_features)
#Missing values in training features: 0
#Missing values in testing features: 0

In [ ]:
# теперь маштабируем [0:1]
# Пример
# Create the scaler object with a range of 0-1
#scaler = MinMaxScaler(feature_range=(0, 1))
# Fit on the training data
#scaler.fit(X)
# Transform both the training and testing data
#X = scaler.transform(X)
#X_test = scaler.transform(X_test)

In [14]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(x)
x = scaler.transform(x)

In [17]:
train_x, test_x, train_y, test_y =  train_test_split(x, y, test_size=0.2, random_state=42)

Наивный байсовский классификатор

In [20]:
gnb = GaussianNB()
y_pred = gnb.fit(train_x, train_y).predict(test_x)
y_pred

array([2, 0, 1, ..., 2, 2, 2], dtype=int64)

In [21]:
f1_score(test_y, y_pred, average='macro')

0.2336022924649217

In [22]:
#result
print("Number of mislabeled points out of a total %d points : %d"
            % (test_x.shape[0], (test_y != y_pred).sum())) 

Number of mislabeled points out of a total 4905 points : 3778


Метод ближайших соседей

In [39]:
#weights возможно с ним поиграть
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(train_x, train_y)

KNeighborsClassifier(n_neighbors=3)

In [40]:
#result
y_pred1=neigh.predict(test_x)
f1_score(test_y, y_pred1, average='macro')
#print(neigh.predict_proba(test_y))

0.39883273829985727

Метод опорных векторов (SVM)

In [77]:
#kernel='sigmoid' пока ручшее 0.3958485583703586 C2.0=0.39626990306811866 C3=0.39308951131700504
#c2.5 0.3977894076748474 2.4 0.39849778171730316
#rbf 0.36202765076937254 poly 0.3634310299900127
clf = svm.SVC(C=2.4, kernel='sigmoid')
clf.fit(train_x, train_y)

SVC(C=2.4, kernel='sigmoid')

In [78]:
#result
y_pred2 = clf.predict(test_x)

f1_score(test_y, y_pred2, average='macro')

0.39849778171730316

Случайный лес/Ансамбль деревьев

In [133]:
#model fit
X, y = make_classification(n_samples=1000, n_features=3,
                           n_informative=2, n_redundant=0,
                           random_state=0, shuffle=False)
clf = RandomForestClassifier(min_samples_split=4, max_features='auto', random_state=0)
clf.fit(train_x, train_y)

RandomForestClassifier(min_samples_split=4, random_state=0)

In [134]:
#result
#print(clf.feature_importances_)
y_pred4 = clf.predict(test_x)

In [135]:
f1_score(test_y, y_pred4, average='macro')

0.40083883513170776

Метод градиентного спуска

In [259]:
#model fit
clf = SGDClassifier(loss="squared_loss", penalty="l2", max_iter=10000, random_state=8, learning_rate='optimal',) #log modified_huber
clf.fit(train_x, train_y)

C:\anaconda3\envs\forkagle\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:573: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


SGDClassifier(loss='squared_loss', max_iter=10000, random_state=8)

In [260]:
#result
y_pred5 = clf.predict(test_x)
clf.coef_
clf.intercept_

array([ 89929763.67444287, -86534773.5298964 ,  36323956.48246042])

In [261]:
f1_score(test_y, y_pred5, average='macro')

0.3097081874128038

### тут надо разбираться в документах и примерах

Градиентный бустинг

In [47]:
#imports
from sklearn import ensemble
from sklearn import datasets
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error

params = {'n_estimators': 500, 'max_depth': 4, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'ls'}
clf = ensemble.GradientBoostingRegressor(**params)
clf.fit(train_x, train_y)

#result
y_pred6 = clf.predict(test_x)
mse = mean_squared_error(test_y, y_pred6)
print("MSE: %.4f" % mse)

MSE: 0.2844


In [274]:
#for row in y_pred6:
#    print(row)

In [278]:
reg = GradientBoostingRegressor(random_state=0)
reg.fit(train_x, train_y)

GradientBoostingRegressor(random_state=0)

In [279]:
reg.predict(test_x)

array([0.18061538, 0.20866926, 0.18252064, ..., 0.07566001, 0.35332276,
       1.22288745])

In [280]:
reg.score(test_x, test_y)

0.1973623990263792

### Это миракле пример

In [266]:
xgb_param = {'objective': 'multi:softprob',
          'eval_metric' : 'mlogloss',
          'learning_rate' : 0.05,
          'max_depth' : 5,
          'num_class' : 9,
          'nthread': 4,
          'seed': 10}

dtrain_xgb = xgb.DMatrix(train_x, label=train_y)

xbg_result = xgb.cv(xgb_param, 
                    dtrain_xgb, 
                    num_boost_round=300, 
                    nfold=3,
                    stratified=True, 
                    early_stopping_rounds=50, 
                    verbose_eval=100, 
                    show_stdv=True)

[0]	train-mlogloss:2.07382+0.00025	test-mlogloss:2.07728+0.00038
[100]	train-mlogloss:0.54281+0.00397	test-mlogloss:0.67790+0.00561
[200]	train-mlogloss:0.44360+0.00658	test-mlogloss:0.64453+0.00704


In [267]:
num_round_xgb = len(xbg_result['test-mlogloss-mean'])
print('num boost rounds xgb=' + str(num_round_xgb))

num boost rounds xgb=241


In [268]:
xgb_cl = xgb.train(xgb_param, dtrain_xgb, num_boost_round=num_round_xgb)


In [272]:
xgtest = xgb.DMatrix(test_x)
y_pred8 = xgb_cl.predict(xgtest)

In [273]:
for row in y_pred8:
    print(row)

[7.9138899e-01 2.0132837e-01 6.3935402e-03 1.4818518e-04 1.4818518e-04
 1.4818518e-04 1.4818518e-04 1.4818518e-04 1.4818518e-04]
[7.7840513e-01 2.1686503e-01 3.2271857e-03 2.5044422e-04 2.5044422e-04
 2.5044422e-04 2.5044422e-04 2.5044422e-04 2.5044422e-04]
[8.1883097e-01 1.7658734e-01 3.6896549e-03 1.4867324e-04 1.4867324e-04
 1.4867324e-04 1.4867324e-04 1.4867324e-04 1.4867324e-04]
[6.9272679e-01 3.0192566e-01 4.2885304e-03 1.7650485e-04 1.7650485e-04
 1.7650485e-04 1.7650485e-04 1.7650485e-04 1.7650485e-04]
[8.5526663e-01 1.4061466e-01 3.2610018e-03 1.4294854e-04 1.4294854e-04
 1.4294854e-04 1.4294854e-04 1.4294854e-04 1.4294854e-04]
[4.3706015e-01 3.8901600e-01 1.7252913e-01 2.3245859e-04 2.3245859e-04
 2.3245859e-04 2.3245859e-04 2.3245859e-04 2.3245859e-04]
[8.8652968e-01 1.0149540e-01 1.1068308e-02 1.5110233e-04 1.5110233e-04
 1.5110233e-04 1.5110233e-04 1.5110233e-04 1.5110233e-04]
[3.30842823e-01 5.54184198e-01 1.13434665e-01 2.56387080e-04
 2.56387080e-04 2.56387080e-04 2.563

 1.8306391e-04 1.8306391e-04 1.8306391e-04 1.8306391e-04]
[7.3790514e-01 2.4051484e-01 2.0570427e-02 1.6826471e-04 1.6826471e-04
 1.6826471e-04 1.6826471e-04 1.6826471e-04 1.6826471e-04]
[9.3992734e-01 5.4972831e-02 4.4516930e-03 1.0801918e-04 1.0801918e-04
 1.0801918e-04 1.0801918e-04 1.0801918e-04 1.0801918e-04]
[7.3539799e-01 1.9569778e-01 6.7585804e-02 2.1973906e-04 2.1973906e-04
 2.1973906e-04 2.1973906e-04 2.1973906e-04 2.1973906e-04]
[4.0216115e-01 4.2843634e-01 1.6749965e-01 3.1714479e-04 3.1714479e-04
 3.1714479e-04 3.1714479e-04 3.1714479e-04 3.1714479e-04]
[8.70131969e-01 6.76178634e-02 6.15256056e-02 1.20768054e-04
 1.20768054e-04 1.20768054e-04 1.20768054e-04 1.20768054e-04
 1.20768054e-04]
[8.9225674e-01 9.9261746e-02 7.7540562e-03 1.2124267e-04 1.2124267e-04
 1.2124267e-04 1.2124267e-04 1.2124267e-04 1.2124267e-04]
[8.2494694e-01 1.5090390e-01 2.2915356e-02 2.0563269e-04 2.0563269e-04
 2.0563269e-04 2.0563269e-04 2.0563269e-04 2.0563269e-04]
[9.2502892e-01 5.8324680e-02 

 2.7214730e-04 2.7214730e-04 2.7214730e-04 2.7214730e-04]
[5.1976472e-01 4.0980363e-01 6.8805516e-02 2.7101624e-04 2.7101624e-04
 2.7101624e-04 2.7101624e-04 2.7101624e-04 2.7101624e-04]
[1.3392158e-01 3.9332789e-01 4.7145993e-01 2.1509806e-04 2.1509806e-04
 2.1509806e-04 2.1509806e-04 2.1509806e-04 2.1509806e-04]
[6.7134321e-01 3.2078522e-01 6.7456616e-03 1.8765326e-04 1.8765326e-04
 1.8765326e-04 1.8765326e-04 1.8765326e-04 1.8765326e-04]
[7.1135503e-01 2.7914202e-01 8.1447614e-03 2.2636779e-04 2.2636779e-04
 2.2636779e-04 2.2636779e-04 2.2636779e-04 2.2636779e-04]
[8.96958530e-01 9.89726111e-02 3.41603206e-03 1.08808475e-04
 1.08808475e-04 1.08808475e-04 1.08808475e-04 1.08808475e-04
 1.08808475e-04]
[8.2555419e-01 1.5802988e-01 1.5413684e-02 1.6704746e-04 1.6704746e-04
 1.6704746e-04 1.6704746e-04 1.6704746e-04 1.6704746e-04]
[2.4901693e-01 3.0748054e-01 4.4200245e-01 2.5000673e-04 2.5000673e-04
 2.5000673e-04 2.5000673e-04 2.5000673e-04 2.5000673e-04]
[8.6571580e-01 1.1213039e-01 

 1.9922364e-04 1.9922364e-04 1.9922364e-04 1.9922364e-04]
[8.2954741e-01 1.5054731e-01 1.9072831e-02 1.3874931e-04 1.3874931e-04
 1.3874931e-04 1.3874931e-04 1.3874931e-04 1.3874931e-04]
[6.2807071e-01 3.6326969e-01 7.5180256e-03 1.9026328e-04 1.9026328e-04
 1.9026328e-04 1.9026328e-04 1.9026328e-04 1.9026328e-04]
[7.3539388e-01 2.5549078e-01 8.2355728e-03 1.4662060e-04 1.4662060e-04
 1.4662060e-04 1.4662060e-04 1.4662060e-04 1.4662060e-04]
[6.6049474e-01 3.1432730e-01 2.3967408e-02 2.0175567e-04 2.0175567e-04
 2.0175567e-04 2.0175567e-04 2.0175567e-04 2.0175567e-04]
[4.7835422e-01 4.1904485e-01 1.0091858e-01 2.8039032e-04 2.8039032e-04
 2.8039032e-04 2.8039032e-04 2.8039032e-04 2.8039032e-04]
[8.8311857e-01 1.1223184e-01 3.5805514e-03 1.7817652e-04 1.7817652e-04
 1.7817652e-04 1.7817652e-04 1.7817652e-04 1.7817652e-04]
[6.0274893e-01 3.1702483e-01 7.8629754e-02 2.6608846e-04 2.6608846e-04
 2.6608846e-04 2.6608846e-04 2.6608846e-04 2.6608846e-04]
[7.9749244e-01 1.9309428e-01 8.4361704e

 1.3972048e-04 1.3972048e-04 1.3972048e-04 1.3972048e-04]
[5.7719457e-01 3.4365693e-01 7.7906094e-02 2.0706355e-04 2.0706355e-04
 2.0706355e-04 2.0706355e-04 2.0706355e-04 2.0706355e-04]
[8.0217427e-01 1.8813652e-01 8.6602392e-03 1.7149906e-04 1.7149906e-04
 1.7149906e-04 1.7149906e-04 1.7149906e-04 1.7149906e-04]
[9.5073622e-01 4.6328776e-02 2.3098902e-03 1.0419511e-04 1.0419511e-04
 1.0419511e-04 1.0419511e-04 1.0419511e-04 1.0419511e-04]
[6.2696254e-01 3.4479684e-01 2.6955720e-02 2.1415252e-04 2.1415252e-04
 2.1415252e-04 2.1415252e-04 2.1415252e-04 2.1415252e-04]
[6.3634127e-01 3.4674600e-01 1.5694015e-02 2.0312140e-04 2.0312140e-04
 2.0312140e-04 2.0312140e-04 2.0312140e-04 2.0312140e-04]
[3.5119492e-01 3.5980472e-01 2.8749737e-01 2.5049329e-04 2.5049329e-04
 2.5049329e-04 2.5049329e-04 2.5049329e-04 2.5049329e-04]
[7.2326708e-01 2.4824290e-01 2.7137388e-02 2.2542951e-04 2.2542951e-04
 2.2542951e-04 2.2542951e-04 2.2542951e-04 2.2542951e-04]
[9.0995216e-01 7.1991585e-02 1.7237907e

...

 1.5322161e-04 1.5322161e-04 1.5322161e-04 1.5322161e-04]
[8.2997257e-01 1.6163482e-01 7.4697775e-03 1.5380535e-04 1.5380535e-04
 1.5380535e-04 1.5380535e-04 1.5380535e-04 1.5380535e-04]
[7.8060800e-01 2.1591124e-01 2.5119814e-03 1.6145690e-04 1.6145690e-04
 1.6145690e-04 1.6145690e-04 1.6145690e-04 1.6145690e-04]
[7.2799045e-01 2.5648019e-01 1.4442521e-02 1.8113019e-04 1.8113019e-04
 1.8113019e-04 1.8113019e-04 1.8113019e-04 1.8113019e-04]
[8.7791806e-01 9.6563734e-02 2.4535840e-02 1.6372661e-04 1.6372661e-04
 1.6372661e-04 1.6372661e-04 1.6372661e-04 1.6372661e-04]
[6.6071439e-01 3.1196639e-01 2.6126662e-02 1.9875745e-04 1.9875745e-04
 1.9875745e-04 1.9875745e-04 1.9875745e-04 1.9875745e-04]
[6.7300755e-01 2.9275170e-01 3.2769937e-02 2.4513944e-04 2.4513944e-04
 2.4513944e-04 2.4513944e-04 2.4513944e-04 2.4513944e-04]
[8.7207544e-01 1.2192472e-01 5.0869263e-03 1.5214473e-04 1.5214473e-04
 1.5214473e-04 1.5214473e-04 1.5214473e-04 1.5214473e-04]
[7.0409465e-01 7.9943404e-02 2.1453674e

 2.3929532e-04 2.3929532e-04 2.3929532e-04 2.3929532e-04]
[7.6058602e-01 2.1754359e-01 2.0281972e-02 2.6474014e-04 2.6474014e-04
 2.6474014e-04 2.6474014e-04 2.6474014e-04 2.6474014e-04]
[7.6109695e-01 2.3159525e-01 6.2558795e-03 1.7532318e-04 1.7532318e-04
 1.7532318e-04 1.7532318e-04 1.7532318e-04 1.7532318e-04]
[7.4093169e-01 2.4457112e-01 1.3384909e-02 1.8537455e-04 1.8537455e-04
 1.8537455e-04 1.8537455e-04 1.8537455e-04 1.8537455e-04]
[6.3432461e-01 3.3015153e-01 3.4037624e-02 2.4770317e-04 2.4770317e-04
 2.4770317e-04 2.4770317e-04 2.4770317e-04 2.4770317e-04]
[7.6494634e-01 2.2240292e-01 1.1552426e-02 1.8305640e-04 1.8305640e-04
 1.8305640e-04 1.8305640e-04 1.8305640e-04 1.8305640e-04]
[8.8907975e-01 1.0495660e-01 5.2361940e-03 1.2123501e-04 1.2123501e-04
 1.2123501e-04 1.2123501e-04 1.2123501e-04 1.2123501e-04]
[7.2895056e-01 2.6705971e-01 2.8807751e-03 1.8481808e-04 1.8481808e-04
 1.8481808e-04 1.8481808e-04 1.8481808e-04 1.8481808e-04]
[8.6685240e-01 1.3035879e-01 1.9719508e

 1.8540799e-04 1.8540799e-04 1.8540799e-04 1.8540799e-04]
[7.6569366e-01 2.2924347e-01 4.1859485e-03 1.4615977e-04 1.4615977e-04
 1.4615977e-04 1.4615977e-04 1.4615977e-04 1.4615977e-04]
[7.3653579e-01 8.7843552e-02 1.7459212e-01 1.7141782e-04 1.7141782e-04
 1.7141782e-04 1.7141782e-04 1.7141782e-04 1.7141782e-04]
[7.52748072e-01 1.37724131e-01 1.08728446e-01 1.33222478e-04
 1.33222478e-04 1.33222478e-04 1.33222478e-04 1.33222478e-04
 1.33222478e-04]
[9.3464196e-01 5.3602215e-02 1.1089212e-02 1.1110097e-04 1.1110097e-04
 1.1110097e-04 1.1110097e-04 1.1110097e-04 1.1110097e-04]
[8.7385929e-01 8.9508794e-02 3.5792757e-02 1.3985834e-04 1.3985834e-04
 1.3985834e-04 1.3985834e-04 1.3985834e-04 1.3985834e-04]
[8.8421631e-01 1.1340466e-01 1.4150165e-03 1.6066704e-04 1.6066704e-04
 1.6066704e-04 1.6066704e-04 1.6066704e-04 1.6066704e-04]
[3.4693187e-01 5.3918707e-01 1.1255536e-01 2.2094104e-04 2.2094104e-04
 2.2094104e-04 2.2094104e-04 2.2094104e-04 2.2094104e-04]
[7.8479815e-01 1.5117551e-01 

 1.5860677e-04 1.5860677e-04 1.5860677e-04 1.5860677e-04]
[8.0989164e-01 1.8687715e-01 2.2073044e-03 1.7066087e-04 1.7066087e-04
 1.7066087e-04 1.7066087e-04 1.7066087e-04 1.7066087e-04]
[7.0096523e-01 2.8790843e-01 9.8398775e-03 2.1440472e-04 2.1440472e-04
 2.1440472e-04 2.1440472e-04 2.1440472e-04 2.1440472e-04]
[3.4801871e-01 2.5211501e-01 3.9884296e-01 1.7054730e-04 1.7054730e-04
 1.7054730e-04 1.7054730e-04 1.7054730e-04 1.7054730e-04]
[7.3901278e-01 2.5682265e-01 3.2717334e-03 1.4880452e-04 1.4880452e-04
 1.4880452e-04 1.4880452e-04 1.4880452e-04 1.4880452e-04]
[8.7212104e-01 1.2283161e-01 4.2116907e-03 1.3926475e-04 1.3926475e-04
 1.3926475e-04 1.3926475e-04 1.3926475e-04 1.3926475e-04]
[7.9039961e-01 2.0127952e-01 7.2674467e-03 1.7557248e-04 1.7557248e-04
 1.7557248e-04 1.7557248e-04 1.7557248e-04 1.7557248e-04]
[9.4605881e-01 4.7027469e-02 6.3756555e-03 8.9685491e-05 8.9685491e-05
 8.9685491e-05 8.9685491e-05 8.9685491e-05 8.9685491e-05]
[7.8538799e-01 1.3687256e-01 7.6279618e

 2.4697842e-04 2.4697842e-04 2.4697842e-04 2.4697842e-04]
[8.4907466e-01 1.2610887e-01 2.3772594e-02 1.7397240e-04 1.7397240e-04
 1.7397240e-04 1.7397240e-04 1.7397240e-04 1.7397240e-04]
[7.6781666e-01 2.2908247e-01 2.0979410e-03 1.6715344e-04 1.6715344e-04
 1.6715344e-04 1.6715344e-04 1.6715344e-04 1.6715344e-04]
[9.5714194e-01 1.7531024e-02 2.5042212e-02 4.7482783e-05 4.7482783e-05
 4.7482783e-05 4.7482783e-05 4.7482783e-05 4.7482783e-05]
[3.2256076e-01 3.4808376e-01 3.2818893e-01 1.9443271e-04 1.9443271e-04
 1.9443271e-04 1.9443271e-04 1.9443271e-04 1.9443271e-04]
[8.8465536e-01 1.0145066e-01 1.3079034e-02 1.3582726e-04 1.3582726e-04
 1.3582726e-04 1.3582726e-04 1.3582726e-04 1.3582726e-04]
[7.1150827e-01 2.3724525e-01 5.0270718e-02 1.6262264e-04 1.6262264e-04
 1.6262264e-04 1.6262264e-04 1.6262264e-04 1.6262264e-04]
[7.1184009e-01 2.7173382e-01 1.5220566e-02 2.0091329e-04 2.0091329e-04
 2.0091329e-04 2.0091329e-04 2.0091329e-04 2.0091329e-04]
[7.3198587e-01 2.4502163e-01 2.1842517e

пример https://habr.com/ru/post/491326/

In [281]:
#imports
import numpy as np
import matplotlib.pyplot as plt
from sklearn import ensemble
from sklearn import datasets
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error

#model fit
boston = datasets.load_boston()
X, Y = shuffle(boston.data, boston.target, random_state=13)
X = X.astype(np.float32)
offset = int(X.shape[0] * 0.9)
X_train, y_train = X[:offset], Y[:offset]
X_test, y_test = X[offset:], Y[offset:]
params = {'n_estimators': 500, 'max_depth': 4, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'ls'}
clf = ensemble.GradientBoostingRegressor(**params)
clf.fit(X_train, y_train)

GradientBoostingRegressor(learning_rate=0.01, max_depth=4, n_estimators=500)

In [282]:

#result
mse = mean_squared_error(y_test, clf.predict(X_test))
print("MSE: %.4f" % mse)

MSE: 6.5036


In [284]:
y_test

array([15.6, 28.4, 28.1, 31.2, 13.1, 37.2, 22. , 11.5, 13.8, 39.8, 28.5,
       15.2, 23.8, 19.4, 27.1, 18.9, 17.9, 45.4, 15.6, 21.6, 21.4, 19.9,
       17.8, 23. , 15.4,  8.3, 27. , 36. , 22.8, 17.1, 22.6, 23.9, 17.7,
       31.5,  8.4, 14.5, 13.4, 15.7, 17.5, 15. , 21.8, 18.4, 25.1, 19.4,
       17.6, 18.2, 24.3, 23.1, 24.1, 23.2, 20.6])